##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [ ]:
# import necessary modules.

import google.generativeai as genai

import base64
import json

try:
    # Mount google drive
    from google.colab import drive

    drive.mount("/gdrive")

    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

# Parse the arguments

model = "gemini-1.5-flash"  # @param {isTemplate: true}
contents_b64 = b'W3sicGFydHMiOiBbeyJ0ZXh0IjogIkhlbGxvIn1dfV0='
generation_config_b64 = "e30="  # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

print(json.dumps(contents, indent=4))

## Call `generate_content`

In [10]:
!pip install nest_asyncio vaderSentiment textblob langdetect googletrans aiohttp cachetools numpy google-generativeai PyPDF2 google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


In [13]:
import os
import nest_asyncio
import asyncio
import logging
import time
import PyPDF2
from datetime import datetime, timedelta
from typing import Dict, Any, List
import aiohttp
import numpy as np
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from langdetect import detect
from googletrans import Translator
import google.generativeai as genai
import cachetools
from google.colab import auth
from googleapiclient.discovery import build
import threading
import io

# ✅ Fix Async Issue for Google Colab / Jupyter Notebook
nest_asyncio.apply()

# 🔹 API Keys
GENAI_API_KEY = "YOUR_GEMINI_API_KEY"
COINMARKETCAP_API_KEY = "04cf4b5b-9868-465c-8ba0-9f2e78c92eb1"
NEWS_API_KEY = "968a5e25552b4cb5ba3280361d8444ab"

# 🔹 Configure Google AI Model
genai.configure(api_key=GENAI_API_KEY)

# 🔹 AI Model Settings
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

# 🔹 Logging Configuration
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# 🔹 Google Drive Authentication
auth.authenticate_user()
drive_service = build('drive', 'v3')

# ✅ Read PDFs from Google Drive (Fixing "bytes" issue)
def read_pdfs_from_drive():
    query = "mimeType='application/pdf'"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])

    pdf_text = ""
    for file in files:
        request = drive_service.files().get_media(fileId=file['id'])
        pdf_content = io.BytesIO(request.execute())  # Fixing bytes error
        reader = PyPDF2.PdfReader(pdf_content)

        for page in reader.pages:
            pdf_text += page.extract_text() + "\n"

    return pdf_text

# ✅ Sentiment Analysis (Fixing "coroutine never awaited" issue)
class SentimentAnalyzer:
    def __init__(self):
        self.vader = SentimentIntensityAnalyzer()
        self.translator = Translator()

    async def fetch_news(self, symbol: str) -> List[Dict]:
        async with aiohttp.ClientSession() as session:
            params = {'q': symbol, 'apiKey': NEWS_API_KEY, 'from': (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d'), 'sortBy': 'publishedAt'}
            async with session.get('https://newsapi.org/v2/everything', params=params) as response:
                if response.status == 200:
                    return (await response.json()).get('articles', [])
                return []

    def analyze_sentiment(self, text: str) -> float:
        try:
            lang = detect(text)
            if lang != 'en':
                text = self.translator.translate(text, dest='en').text
            vader_score = self.vader.polarity_scores(text)['compound']
            blob_score = TextBlob(text).sentiment.polarity
            return (vader_score + blob_score) / 2
        except:
            return 0.0

# ✅ Fundamental Analysis
class FundamentalAnalyzer:
    def __init__(self):
        self.headers = {'X-CMC_PRO_API_KEY': COINMARKETCAP_API_KEY, 'Accept': 'application/json'}

    async def fetch_market_data(self, symbol: str) -> Dict:
        async with aiohttp.ClientSession() as session:
            url = f"https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest"
            params = {'symbol': symbol, 'convert': 'USD'}
            async with session.get(url, params=params, headers=self.headers) as response:
                if response.status == 200:
                    return (await response.json()).get("data", {}).get(symbol, {})
                return {}

# ✅ AI Crypto Analysis & Continuous Learning
async def analyze_crypto(symbol: str):
    sentiment_analyzer = SentimentAnalyzer()
    fundamental_analyzer = FundamentalAnalyzer()

    # 🔹 Fetch News & Sentiment Analysis
    news_articles = await sentiment_analyzer.fetch_news(symbol)
    sentiment_scores = [sentiment_analyzer.analyze_sentiment(article.get('title', '') + ' ' + article.get('description', '')) for article in news_articles]
    sentiment_score = np.mean(sentiment_scores) if sentiment_scores else 0.0

    # 🔹 Fetch Fundamental Data
    market_data = await fundamental_analyzer.fetch_market_data(symbol)
    price = market_data.get("quote", {}).get("USD", {}).get("price", 0)

    # 🔹 Fetch PDF Data from Google Drive
    pdf_text = read_pdfs_from_drive()

    # 🔹 AI Model for Price Prediction
    model = genai.GenerativeModel(model_name="tunedModels/model-predicts-the-future-price-of-crypt", generation_config=generation_config)

    prompt = f"""
    Predict the price of {symbol.upper()} for the next 24 hours based on historical market trends,
    including open, high, low, close, volume, and market capitalization.
    Also, provide insights on potential bullish or bearish movements and trading opportunities.
    Current market price: {price}
    Sentiment Score: {sentiment_score}

    Additionally, use the following trading strategies and analysis from the latest PDFs:
    {pdf_text}
    """

    response = model.generate_content(prompt)

    # 🔹 Output Analysis Report
    print(f"\n🔹 Analysis Report for {symbol.upper()}")
    print("=" * 50)
    print(f"📊 Current Price: ${price:.2f}")
    print(f"💬 Sentiment Score: {sentiment_score:.2f}")
    print("\n📈 Predicted Price and Market Insights:")
    print(response.text)

# ✅ Continuous Learning Every Hour
def continuous_learning():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    symbols = ["BTC", "ETH", "XRP"]
    for symbol in symbols:
        loop.run_until_complete(analyze_crypto(symbol))

# ✅ Run Continuous Learning in a Background Thread
def start_background_learning():
    learning_thread = threading.Thread(target=continuous_learning, daemon=True)
    learning_thread.start()

# ✅ Start Learning Every Hour Without Blocking Execution
start_background_learning()


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>